## MCC

The goal of this notebook is to evaluate simple classification approaches on profile data.  Can we learn a classifier to label profiles with the MCC based on some labeled data?

My process will be:
* Do a little exploration - what does the data look like?  What labeled data do I have?
* Build a crappy pipeline from data pre-processing to evaluating a classifier with a confusion matrix
* Try to build a decent word feature extractor to see if that will improve performance

This notebook starts the exploratory analysis.

In [3]:
import pandas as pd
from collections import defaultdict
base_dir='c:\\Users\\sgoldman\\MCC\\'

In [32]:
#basic word frequency analysis from detailed composition data

word_counts=defaultdict(int)

with open(base_dir+'Profile_Composition_6-29-2017.txt') as f:
    for record in f.readlines()[2:]:
        text=record[87:138]
        for word in text.split():
            word_counts[word]+=1

print 'There are %d words\n'%len(word_counts)

print 'Top words:'
for f,word in sorted([(v,k) for (k,v) in word_counts.iteritems()],reverse=True)[:1]:
    print f, word

There are 22072 words

Top words:
16180 SODIUM


In [30]:
# basic word frequency analysis from profile data (standard metals not included)
# the composition data has already been processed by the data extraction SQL query

word_counts=defaultdict(int)

with open(base_dir+'Profile_Data_6-29-2017.txt') as f:
    #for idx,field in enumerate(f.readlines()[0].split('|')):
    #    print idx,field
    for record in f.readlines()[1:]:
        text=record.split('|')[22]
        for word in text.split():
            word_counts[word]+=1

print 'There are %d words\n'%len(word_counts)

print 'Top words:'
for f,word in sorted([(v,k) for (k,v) in word_counts.iteritems()],reverse=True)[:200]:
    print f, word

There are 18388 words

Top words:
5881 OF:
5695 WATER
4932 CONSISTING
3613 NON-HALOGENATED
2843 SOLVENTS:
2773 ACID
2568 ETHYL
2196 SODIUM
2191 SMALL
2176 CONTAINERS
2117 METHYL
1885 ACETONE,
1575 TOLUENE,
1499 OIL
1429 PETROLEUM
1376 IN
1298 METHANOL,
1203 ACETATE,
1202 ETHANOL,
1190 PAPER,
1188 PPE,
1176 CHLORIDE
1117 CONTAINING:
1109 XYLENE,
1056 AND
1043 CARBON
1006 KETONE,
1003 PLASTIC,
983 OF
978 MINERAL
974 OIL,
942 BENZENE
925 PAINT
923 PLASTIC
920 DEBRIS
899 RAGS,
891 OR
832 HYDROXIDE
811 GLYCOL
794 METHANOL
767 POTASSIUM
737 TITANIUM
728 SOLVENTS
727 DISTILLATES,
704 RESIN
686 METHYLENE
659 ***********************************
642 ISOPROPANOL,
623 CALCIUM
623 BE
617 MAY
611 XYLENE
607 HALOGENATED
604 SOLIDS
585 TOLUENE
584 DEBRIS:
565 ACETIC
557 LATEX
555 DIOXIDE
543 CLAY,
528 PEROXIDE
528 NOT
528 ACID,
497 ACETONITRILE
492 ETHER
487 PAPER
486 BASED
486 ALUMINUM
486 ACETATE
472 ***
471 PADS,
466 ETHYLENE
464 AMMONIUM
453 DIRT,
447 OXIDE
447 ORGANICS:
447 METAL
446 ETHANOL
442 

In [9]:
# 
profiles=pd.read_csv(base_dir+'Profile_Data_6-29-2017.txt',sep='|')
profiles.info()
#profiles['Matl Category Code'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13573 entries, 0 to 13572
Data columns (total 31 columns):
WPS                     13573 non-null int64
EPA Waste Name          13573 non-null object
Waste Material Name     13573 non-null object
Matl Category Code      13383 non-null object
Physical Description    13573 non-null object
RCRA                    13573 non-null object
DOT Haz Class           13573 non-null object
Lab Pack/Loospack/No    13573 non-null object
SWRC Qty Restriction    4611 non-null object
Low BTU                 13573 non-null int64
High BTU                13573 non-null int64
Low PH                  13573 non-null float64
High PH                 13573 non-null float64
High ASH                13573 non-null int64
total halogen-low       13478 non-null float64
total halogen-high      13478 non-null float64
Physical States         13573 non-null object
CSF                     13573 non-null object
CSF Storage Code        13573 non-null int64
Reactive           

In [8]:
# label counts - eliminate all non-numerics
label_counts=defaultdict(int)

with open(base_dir+'Profile_Data_6-29-2017.txt') as f:
    
    for record in f.readlines()[1:]:
        label=record.split('|')[3].strip()
        if not label.isdigit():
            label='UNLABELED'
        label_counts[label]+=1

print 'There are %d labels\n'%len(label_counts)

print 'Labels:'
for f,label in sorted([(v,k) for (k,v) in label_counts.iteritems()],reverse=True):
    print label,f

There are 139 labels

Labels:
UNLABELED 9489
7001 405
9502 378
7700 296
7401 165
7010 156
5501 112
9505 109
6501 105
9401 98
4001 97
7006 92
7003 91
9501 89
9201 79
7008 77
9050 72
9009 67
3001 63
9008 59
7103 55
7007 52
3002 50
8003 44
7503 43
9006 41
6301 41
8201 38
7012 38
7501 37
5201 34
9007 33
9552 31
2103 30
1101 29
3004 28
7504 27
5502 27
8002 25
9799 24
9011 24
7005 24
9251 23
9004 19
7502 18
3003 18
2001 18
8202 17
9001 16
1103 16
1009 16
1001 16
7101 15
7004 15
9699 14
9405 14
9002 14
7402 14
6002 14
5005 14
1005 14
9801 13
9451 13
9010 13
9003 13
6503 13
6001 13
4003 13
1002 13
9800 12
6201 12
9452 11
1003 11
9900 10
9802 10
5007 9
4004 9
8203 8
7011 8
4009 8
9553 7
8001 7
6204 7
5001 7
1102 7
9803 6
9551 6
7505 6
5004 6
8301 5
7702 5
7009 5
6502 5
5006 5
4002 5
3102 5
9503 4
9013 4
8303 4
8004 4
5401 4
3006 4
9406 3
9400 3
9101 3
9052 3
7102 3
6202 3
5801 3
3005 3
9991 2
9402 2
8302 2
7057 2
6601 2
6205 2
5802 2
5002 2
3101 2
1007 2
1006 2
9990 1
9542 1
9504 1
9301 1
9005 